## Spider

Spider alternatifi çok sayıda sayfayı indirmek istediğimizde daha hızlı çalışıyor. Bunun için sınıf ismini verdiğimiz nesnelere kısa bir göz atalım. Nesne tabanlı bir programlama dili olluğu için Python'da neredeyse her şey bir sınıf olarak tasarlanır. Sınıf (class) kendisinden örnek oluşturulabilmesi ve iç içe fonksiyonlar çalıştırabilmesi yönüyle programlama için çok önemli bir araçtır. Sınıflara çeşitli özellik ve yöntemler atanabilir.

#### Nasıl çalışır?

Aşağıdaki kodda IMDB_Spider isimli bir sınıf tanımlayabilirsiniz. Bu sınıf scrapy kütüphanesinin Spider metodunu çalıştırıyor. İçerikteki ```start_requests``` fonksiyonu ilk olarak işlemi başlatmaya yarıyor. Bu aşamada indirme işleminin başlayacağı ilk sayfayı tanımlayabilirsiniz. Ardından class boyunca modifiye edilecek olan ve fonksiyondan fonksiyona tanışan  ```self``` parametresine gerekli müdahaleyi yaparak bir sonraki fonksiyona (```parse_front```) geçiş sağlayın.

```Parse_front``` fonksiyonu ile, bir önceki fonksiyonda URL'sini tanımlayarak giriş yaptığınız yerdesiniz. Şimdi kullanılacak olan Xpath'lerinizi tanımlayacak, açacak ve belli listeler halinde saklayacaksınız. Bu aşamada ilerlemek istediğiniz linklerinizi de oluşturun. Bu linkleri aşağıdaki komutu kullanarak bir sonraki fonksiyona taşıyabilirsiniz.

```yield response.follow(url = url,  callback = self.parse_pages)```

Artık ```parse_pages``` fonksiyonuna kadar geldiniz. Bu fonksiyon bir önceki aşamada yarattığınız ve buraya ilettiğiniz her bir URL için belirlediğiniz işlemi yapacak. Yani tekrar bir Xpath tanımlayıp çekmek istediğiniz veriyi işaretleyebilirsiniz. Ardından indirdiğiniz veriyi saklayarak sınıfı terk ediniz.

Şimdi tanımladığınız bu sınıfı çalıştırmak için yapmanız gerek onu çağırmak:


```process = CrawlerProcess()```

```process.crawl(IMDB_Spider)```

```process.start()```

Bunu yapmadan önce boş listeler tanımlamayı unutmayın, çünkü indirdiğiniz verileri bu boş listelere yazacaksınız.

In [ ]:
import scrapy
from scrapy.crawler import CrawlerProcess

class IMDB_Spider(scrapy.Spider):
    name = "IMDB_Spider"

    def start_requests(self):
        yield scrapy.Request(url = 'https://m.imdb.com/chart/top', callback = self.parse_front)

    def parse_front(self, response):
        
        movie_names = response.xpath('//h4/text()').extract()
        for item in movie_names:
            cleaned_string = item.strip()
            if cleaned_string != '':
                movie_list.append(cleaned_string)
        
        movie_years = response.xpath('//h4/span[2]/text()').extract()
        for item in movie_years:
                years.append(item)

        movie_links = response.xpath('//*[@id="chart-content"]/div/div/div/a/@href').extract()
        for item in movie_links:
            first_part_url = 'https://m.imdb.com'
            last_part_url = '?pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=df09bbba-7a44-41c0-bc85-426ba05a5574&pf_rd_r=R9N3Q0473JZET8YH4S2A&pf_rd_s=top-1&pf_rd_t=15506&pf_rd_i=top&ref_=m_chttp_tt_1'
            url = first_part_url + item + last_part_url
            links.append(url)
            yield response.follow(url = url,  callback = self.parse_pages)
            
    def parse_pages(self, response):
        movie_players = response.xpath('//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/section[4]/div[2]/div[2]/div/div[2]/a/text()').extract()
        players_list.append(movie_players)


movie_list = []
years = []
links = []
players_list = []

process = CrawlerProcess()
process.crawl(IMDB_Spider)
process.start()

### İlk beş satırı görüntüle

In [ ]:
links[0:5]

In [ ]:
years[0:5]

In [ ]:
movie_list[0:5]

In [ ]:
players_list[0:5]

### Önce dataframe, ardından csv veya json

Elinizdeki veriyi önce bir sözlüğe ardından bir pandas DataFrame'ine dönüştürün.

In [ ]:
import pandas as pd

dictimdb = {'movie name':movie_list, 'year':years, 'link':links, 'player_list': players_list}
data_imdb = pd.DataFrame(dictimdb)

In [ ]:
data_imdb

In [ ]:
data_imdb['player_list'][56]

Şimdi de ilgili veriyi csv veya json dosyası olarak bilgisayarınıza kaydedebilirsiniz.

In [ ]:
data_imdb.to_csv("IMDB_Filmlerim.csv")
data_imdb.to_json("IMDB_Filmlerim.json")